In [1]:
from engine.torchengine import AnalyticalSetSympy, FunctionSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, ElimResidual
from engine.torchdata import symbols, generate_optim_functions
import numpy as np
import sympy as sp
import torch
np.set_printoptions(formatter={'float': lambda x: 
                                f"{x:.3f}".rstrip('0').rstrip('.')})

In [2]:
Pt, Pbus, S, D, A, R, indices = symbols('P_t,P_bus,S,D,A,R', dim=1)
eta_t, eta_a, alpha, th, W0, Re, mu, c1, c2, c3, c4 = 0.16, 0.5, 1e20, np.pi/4, 1367, 6378, 3.986005e14, 2500, 12000, 100, 12000

In [ ]:
h = [
    A*eta_a*W0*sp.cos(th)-Pbus == Pt,
    10*abs(Pt)**0.5 == Pbus,
    alpha*Pt*D**2*eta_t/(16*S**2)*1/(8e6) == R,
    R == 1
]
newmodel = presolve(model, eliminate=[1])
newmodel.solve()

In [ ]:
h = [
    A*eta_a*W0*sp.cos(th)-Pbus == Pt,
    10*abs(Pt)**0.5 == Pbus,
    alpha*Pt*D**2*eta_t/(16*S**2)*1/(8e6) == R,
    R == 1
]
newmodel = presolve_merge(model, [1,2], [3,4], [5]) #groups statevars; transforms a_i
newmodel = presolve(newmodel, eliminate=[1,4,5]) # eliminate state vars that are not shared; there might be a shared var for 2 and 3, so we don't eliminate these
newmodel = presolve_merge(model, [2, 3]) # groups shared vars, "MDF"
newmodel.solve() # this becomes and MDF solve

In [ ]:
h = [
    Discipline1(),
    Discipline2(),
    x == 1
]
newmodel, mdfconstraint = presolve_mdf(model)
newmodel = presolve(newmodel, eliminate=[mdfconstraint])
newmodel.solve()

In [3]:
a1 = AnalyticalSetSympy(A*eta_a*W0*sp.cos(th)-Pbus, Pt, indices)
a2 = AnalyticalSetSympy(10*abs(Pt)**0.5, Pbus, indices)
a3 = AnalyticalSetSympy(alpha*Pt*D**2*eta_t/(16*S**2)*1/(8e6), R, indices)
h1 = FunctionSympy(-R+1, indices)
objf = FunctionSympy((c1*D**2+c2*A+c3*Pbus+c4)*1e-3, indices)

### Elimination Manual

In [11]:
S0 = EliminateAnalysisMergeResiduals(functions=[a2.residual, a3.residual])
F0 = EliminateAnalysis([a1.analysis], [objf, S0, h1])

In [55]:
x0 = torch.tensor([10,10,400e3,0.67,0.1,4.453],dtype=torch.float64)
bounds = [(-np.inf,np.inf), (-np.inf,np.inf), (0,np.inf), (0.01, 10), (0.01, 10), (-np.inf,np.inf)]
solver_indices = [1,2,4,5]
xguess, obj, ineq, eq, dobj, dineq, deq = generate_optim_functions(
    F0, solver_indices, x0, inequality_direction='positive-null', 
    residuals=True, inequalities=False)
constraints = [{'type': 'eq', 'fun': eq, 'jac': deq}]

In [56]:
xguess

array([10, 400000, 0.1, 4.453])

In [57]:
from scipy import optimize
optimize.minimize(obj, xguess, jac=dobj, constraints=constraints,
                  bounds=[bounds[idx] for idx in solver_indices])

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 15.300925010403073
       x: [ 1.689e+01  4.000e+05  4.084e-02  1.000e+00]
     nit: 3
     jac: [ 1.000e-01  0.000e+00  1.200e+01  0.000e+00]
    nfev: 3
    njev: 3

### MDF structure 1

In [4]:
S1 = EliminateAnalysisMergeResiduals(functions=[a1.residual, a2.residual]) # Newton / Jacobi
E1 = ElimResidual(S1, solvefor=[a1.outputvar, a2.outputvar], indices=indices)
F1 = EliminateAnalysis([E1, a3.analysis], [objf, h1])

In [5]:
x0 = torch.tensor([10,10,400e3,0.67,0.1,4.453],dtype=torch.float64)
solver_indices = torch.tensor([3,4])
xguess, obj, ineq, eq, dobj, dineq, deq = generate_optim_functions(
    F1, solver_indices, x0, inequality_direction='positive-null', 
    residuals=True, inequalities=False)
constraints = [{'type': 'eq', 'fun': eq, 'jac': deq}]

In [6]:
from scipy import optimize
optimize.minimize(obj, xguess, jac=dobj, constraints=constraints,
                  bounds=[(0.01, 10), (0.01, 10)])

c:\Users\johan\miniconda3\envs\minimdo\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 15.30092320807752
       x: [ 6.705e-01  4.080e-02]
     nit: 9
     jac: [ 3.352e+00  4.814e+01]
    nfev: 9
    njev: 9

In [7]:
x0.detach().numpy()

array([2.847, 16.874, 400000, 0.67, 0.041, 4.453])

In [8]:
objf(x0).detach().numpy()

array([15.301])